# Download GAIA stars

To be used as filler targets, generate a list of GAIA stars around a given pointing

In [0]:
L, B = 90.0, 28.0
RAD = 60            # arc min
DATA_FILE = '/datascope/subaru/data/targeting/MW/outerdisk_l90_b28_SSP/ga_targets_outerdisk_l90_b28_faint.ecsv'
OUTPUT_FILE = '/datascope/subaru/data/targeting/MW/outerdisk_l90_b28_SSP/l90b28_gaia.feather'

In [0]:
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import SkyCoord

In [0]:
# Convert input galactic coordinates to equatorial
l, b = L * u.deg, B * u.deg
gal = SkyCoord(l=l, b=b, frame='galactic')
eq = gal.transform_to('icrs')
eq.ra.deg, eq.dec.deg

### Load input target list

In [0]:
from pfs.ga.targeting.io import ObservationSerializer

In [0]:
reader = ObservationSerializer()
reader.column_map = {
    'ra': 'RA',
    'dec': 'Dec',
}
obs = reader.read(DATA_FILE)
obs.data.shape, obs.data.columns

### Run cone search on the GAIA database

In [0]:
from pfs.ga.targeting.io import GaiaReader

In [0]:
reader = GaiaReader()
gaia = reader.cone_search(eq, RAD * u.arcmin)
gaia.data.shape, gaia.data.columns

In [0]:
type(gaia)

In [0]:
mask = (gaia.data['gaia_g'] > 21.5) & \
    (gaia.data['gaia_g'] < 22.5) & \
    (gaia.data['gaia_g'] - gaia.data['gaia_rp'] > 0.0) & \
    (gaia.data['gaia_g'] - gaia.data['gaia_rp'] < 2.0)

gaia_filt = gaia.filter(mask=mask)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(4, 6))

for g, c in zip([gaia, gaia_filt], ['lightgray', 'red']):
    ax.plot(g.data['gaia_g'] - g.data['gaia_rp'], g.data['gaia_g'], '.')

ax.invert_yaxis()

### Plot

In [0]:
from pfs.ga.targeting.instrument import SubaruWFC, SubaruPFI
from pfs.ga.targeting.diagram import CMD, CCD, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import Pointing, WcsProjection
from pfs.ga.targeting.instrument import SubaruWFC, SubaruPFI
from pfs.ga.targeting.diagram import FOV, FP

In [0]:
pointing = Pointing(eq, posang=0.0)
print(pointing.ra, pointing.dec)

wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(wcs.pointing)
fov = FOV(projection=wcs)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=240, subplot_kw=dict(projection=fov.projection.wcs))

fov.plot_catalog(ax, obs, c='blue', size=0.5) #, scalex=False, scaley=False)
fov.plot_catalog(ax, gaia_filt, c='red', size=0.5) #, scalex=False, scaley=False)

pfi = SubaruPFI(SubaruWFC(pointing))
pfi.plot_focal_plane(ax, fov, corners=True, color='k', lw=2, scalex=False, scaley=False)

ax.grid()

In [0]:
writer = ObservationSerializer()
writer.write(gaia_filt, OUTPUT_FILE)